In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize_scalar

In [2]:
# read data from 'Data2019PS2.csv'
data = pd.read_csv('Data2019PS2.csv', header=None)
data[0] = data[0].astype(int)
data[1] = data[1].astype(int)

In [3]:
# products' charasteristics
X = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.10]

## (a)

First, we define the function which makes `pd.DataFrame` instance with market share of each products and market share of out side options in each market.
This method enables us to manipulate data easily.

In [4]:
def create_data(data, population):
    """
    Return a `pd.DataFrame` instance with the columns of 'Characteristics', 'Market Share Ratio'
    and 'OutSide Share Ratio'.
    
    Parameters
    ========
    data : pd.DataFrame
        data used
        
    population : scalar(int)
        Population in each market
        
    Returns
    ======
    : pd.DataFrame
    """
    num_products = len(data[0])
    num_market = len(np.unique(data.loc[:,0].values))
    DATA = pd.DataFrame({'Market ID' : data[0],
                                            'Product ID' : data[1],
                                            'Market Share' : data[2],
                                            'Characteristics' : np.ones(num_products),
                                            'Market Share Ratio' : data[2] / population,
                                            'OutSide Share Ratio' : np.ones(num_products)})
    for i in range(num_products):
        DATA.loc[i, 'Characteristics'] = X[DATA.loc[i, 'Product ID']-1]
    outside_share = np.empty(num_market)
    for m in range(1, num_market+1):
        market_data = DATA[DATA['Market ID'] == m]
        outside_share[m-1] = 1 - sum(market_data.loc[:,'Market Share Ratio'].values)
    for i in range(num_products):
        DATA.loc[i,'OutSide Share Ratio'] = outside_share[DATA.loc[i,'Market ID']-1]
    return DATA

In [5]:
# Create new `pd.DataFrame` instance, where population of each market is 1000000
data_a = create_data(data, 1000000)

In [6]:
data_a.head()

,Market ID,Product ID,Market Share,Characteristics,Market Share Ratio,OutSide Share Ratio
0,1,1,127379.0,0.01,0.127379,0.045863
1,1,2,128477.0,0.02,0.128477,0.045863
2,1,3,132527.0,0.03,0.132527,0.045863
3,1,4,136916.0,0.04,0.136916,0.045863
4,1,6,139476.0,0.06,0.139476,0.045863


Then, run OLS regression.

In [7]:
x = np.stack([np.ones(len(data_a['Market ID'])),
                        np.asarray(data_a.loc[:,'Characteristics'].values)],
                       axis=1)
y = np.log(data_a.loc[:,'Market Share Ratio'].values) - np.log(data_a.loc[:,'OutSide Share Ratio'].values)
res = np.linalg.solve(np.dot(x.T, x), np.dot(x.T, y))
res

array([0.99833562, 1.97260981])

Next, let's estimate the  variance-covariance matrix.

In [8]:
residual = y - np.dot(x, res)
var_cov = np.dot(np.dot(residual, residual), np.linalg.inv(np.dot(x.T, x))) / (len(data_a['Market ID']) - len(res))
se1, se2 = np.sqrt(var_cov[0,0]), np.sqrt(var_cov[1,1])

In [9]:
print(se1)
print(se2)

0.00022753322270299025
0.003687366082569901


Thus, the estimated parameters are

\begin{eqnarray*}
    \beta_0 &=& 0.99833562 \ (se = 0.00022753322270299025) \\
    \beta_1^m &=& 1.97260981 \  (se = 0.003687366082569901)
\end{eqnarray*}

## (b)

We can estimate $\beta_0$ and $\beta_1^m$ by same procedure as in (a).

In [10]:
# Create new `pd.DataFrame` instance, where population of each market is 2000000
data_b1 = create_data(data, 2000000)

In [11]:
data_b1.head()

,Market ID,Product ID,Market Share,Characteristics,Market Share Ratio,OutSide Share Ratio
0,1,1,127379.0,0.01,0.063689,0.522932
1,1,2,128477.0,0.02,0.064239,0.522932
2,1,3,132527.0,0.03,0.066264,0.522932
3,1,4,136916.0,0.04,0.068458,0.522932
4,1,6,139476.0,0.06,0.069738,0.522932


In [12]:
# Run OLS regression
x = np.stack([np.ones(len(data_b1['Market ID'])),
                        np.asarray(data_b1.loc[:,'Characteristics'].values)],
                       axis=1)
y = np.log(data_b1.loc[:,'Market Share Ratio'].values) - np.log(data_b1.loc[:,'OutSide Share Ratio'].values)
res = np.linalg.solve(np.dot(x.T, x), np.dot(x.T, y))
res

array([-2.16959252,  1.85755471])

In [13]:
# Compute standard error
residual = y - np.dot(x, res)
var_cov = np.dot(np.dot(residual, residual), np.linalg.inv(np.dot(x.T, x))) / (len(data_a['Market ID']) - len(res))
se1, se2 = np.sqrt(var_cov[0,0]), np.sqrt(var_cov[1,1])

In [14]:
print(se1)
print(se2)

0.0048152409225380175
0.07803500449842635


Therefore, estimated parameters are

\begin{eqnarray*}
    \beta_0 &=& -2.16959252 \  (se = 0.0048152409225380175) \\
    \beta_1^m &=& 1.85755471 \  (se = 0.07803500449842635)
\end{eqnarray*}

In [15]:
# Create new `pd.DataFrame` instance, where population of each market is 4000000
data_b2 = create_data(data, 4000000)

In [16]:
data_b2.head()

,Market ID,Product ID,Market Share,Characteristics,Market Share Ratio,OutSide Share Ratio
0,1,1,127379.0,0.01,0.031845,0.761466
1,1,2,128477.0,0.02,0.032119,0.761466
2,1,3,132527.0,0.03,0.033132,0.761466
3,1,4,136916.0,0.04,0.034229,0.761466
4,1,6,139476.0,0.06,0.034869,0.761466


In [17]:
# Run OLS regression
x = np.stack([np.ones(len(data_b2['Market ID'])),
                        np.asarray(data_b2.loc[:,'Characteristics'].values)],
                       axis=1)
y = np.log(data_b2.loc[:,'Market Share Ratio'].values) - np.log(data_b2.loc[:,'OutSide Share Ratio'].values)
res = np.linalg.solve(np.dot(x.T, x), np.dot(x.T, y))
res

array([-3.23914134,  1.85350799])

In [18]:
# Compute standard error
residual = y - np.dot(x, res)
var_cov = np.dot(np.dot(residual, residual), np.linalg.inv(np.dot(x.T, x))) / (len(data_a['Market ID']) - len(res))
se1, se2 = np.sqrt(var_cov[0,0]), np.sqrt(var_cov[1,1])

In [19]:
print(se1)
print(se2)

0.0049696693169265635
0.08053764572544997


Therefore, estimated parameters are

\begin{eqnarray*}
    \beta_0 &=& -3.23914134 \  (se = 0.0049696693169265635) \\
    \beta_1^m &=& 1.85350799 \  (se = 0.08053764572544997)
\end{eqnarray*}

From all these estimations, we can see that when we change the size of population, $\beta_1^m$ is not changed. However as we increase the number of population, $\beta_0$ decreases. The larger population implies that the market share of outside option becomes large. In this framework, this means that the probability of choosing the product in the market decreases and the mean utility decreases. Hence, as the size of population becomes large, the level of mean utility declines, $\beta_0$ decreases. However, the marginal utility of each products is not changed because outside option does not affect the relative probability of choosing each products, so $\beta_1^m$ remains same value. 

## (c)

First, create `BLP` class in order to handle the data easily.

In [20]:
class BLP:
    """
    Class representing the BLP random coeeficient model.
    All methods are suited for this homework, not BLP model in general.
    
    Parameters
    ========
    data : pd.DataFrame instance
        The data we manipulate. The data must have the column named 'Market ID', 'Characteristics',
        'Market Share Ratio' and 'OutSide Share Ratio'.
        Input `data` can be created by `create_data` method.
        
    consumers : scalar(int)
        The number of consumers, or simulations.
        
    Attributes
    =======
    num_products : scalar(int)
        The number of products in the whole market.
        
    market_id : ndarray(int, ndim=1)
        The array representing the market id of each product.
        
    num_market : scalar(int)
        The number of markets.
        
    consumers : scalar(int)
        See Parameters.
        
    characteristics : ndarray(float, ndim=1)
        The array representing the characteristics of each product.
        
    share_ratio : ndarray(float, ndim=1)
        The array representing the market share of each products.
        
    outside_ratio : ndarray(float, ndim=1)
        The array representing the market share of outside option in the market of each products.
        
    nus : ndarray(float, ndim=1)
        The array of random numbers, which follow standard normal distribution.
        
    delta : ndarray(float, ndim=1)
        The array representing the optimal deltas.
    """
    def __init__(self, data, consumers, seed=1):
        self.num_products = len(data['Market ID'])
        self.market_id = data.loc[:,'Market ID'].values
        self.num_markets = len(np.unique(self.market_id))
        self.consumers = consumers
        self.characteristics = data.loc[:,'Characteristics'].values
        self.share_ratio = np.log(data.loc[:,'Market Share Ratio'].values)
        self.outside_ratio = np.log(data.loc[:,'OutSide Share Ratio'].values)
        self.nus = np.random.RandomState(seed).randn(consumers)  # standard normal distribution
        self.delta = np.empty(self.num_products)
        
    def _compute_deltas(self, deltas):
        mat_deltas = np.empty((self.num_products, self.consumers))
        for col in range(self.consumers):
            mat_deltas[:, col] = deltas[:]
        return mat_deltas
    
    def _compute_mus(self, beta):
        mus = np.empty((self.num_products, self.consumers))
        for row in range(self.num_products):
            for col in range(self.consumers):
                mus[row, col] = self.characteristics[row] * beta * self.nus[col]
        return mus

    def _compute_sjs(self, deltas, mus):
        exp_mat = np.exp(deltas + mus)
        mat = np.empty((self.num_products, self.consumers))
        for m in range(1, self.num_markets+1):
            boollist = self.market_id == m
            mat[boollist, :] = exp_mat[boollist, :] / (np.ones(self.consumers) + np.sum(exp_mat[boollist,:], axis=0))
        sjs = np.mean(mat, axis=1)
        return np.log(sjs)
    
    def compute_opt_delta(self, beta, max_iter=10000, tol=1e-6):
        """
        The method which returns optimal deltas by contraction mapping
        
        Parameters
        ========
        beta : scalar(float or int)
            Parameter used in creating the matrix of µ`s
            
        Return
        =====
        d_old : ndarray(float)
            Optimal delta
        """
        mus = self._compute_mus(beta)
        d_old = self.share_ratio - self.outside_ratio
        d_new = np.empty(self.num_products)
        distance = 1
        for i in range(max_iter):
            d_new[:] = d_old + self.share_ratio - self._compute_sjs(self._compute_deltas(d_old), mus)
            distance = np.dot(d_new-d_old, d_new-d_old)
            d_old[:] = d_new[:]
            if distance < tol:
                break

        return d_old
    
    def opt_zeta(self, beta):
        """
        The method which returns the GMM estimter. In this homework, GMM estimator
        equals to the inner products of the OLS residuals.
        
        Parameters
        ========
        beta : scalar(float or int)
            Parameter used in computing optimal delta
            
        Attributes
        =======
        : ndarray(float, ndim=1)
            The GMM estimater
        """
        deltas = self.compute_opt_delta(beta)
        x = np.stack([np.ones(self.num_products), self.characteristics], axis=1)
        beta_hat = np.linalg.solve(np.dot(x.T, x), np.dot(x.T, deltas))
        residuals = deltas - np.dot(x, beta_hat)
        self.delta = deltas
        return np.dot(residuals, residuals)
    
    def opt_beta(self):
        """
        The method which returns the optimal value of parameter by nonlinear search.
        
        Return
        =====
        result : ndarray(float, ndim=1)
            The estimated parameter.
        """
        result = minimize_scalar(self.opt_zeta)
        return result

First, we create `BLP` instance.

In [21]:
blp = BLP(data_a, 1000)

Let's estimate the random coefficients model.

In [22]:
%%time
beta_u = blp.opt_beta()

CPU times: user 3min 36s, sys: 11.2 s, total: 3min 48s
Wall time: 3min 47s


In [23]:
# estimated \beta_1^u
beta_u

     fun: 0.5585584194788649
    nfev: 21
     nit: 17
 success: True
       x: 1.087871182595932

In [24]:
# run OLS regresion again, and get estimated \beta_0 and \beta_1^m
x = np.stack([np.ones(len(data_a['Market ID'])),
                        np.asarray(data_a.loc[:,'Characteristics'].values)],
                       axis=1)
y = np.log(data_a.loc[:,'Market Share Ratio'].values) - np.log(data_a.loc[:,'OutSide Share Ratio'].values)
res = np.linalg.solve(np.dot(x.T, x), np.dot(x.T, blp.delta))
res

array([0.99972146, 1.93186667])

Therefore, estimated parameters are

\begin{eqnarray*}
    \beta_0 &=& 0.99972146 \\
    \beta_1^m &=& 1.93186667 \\
    \beta_1^u &=& 1.087871182595932
\end{eqnarray*}

## (d)

We can estimate parameters by same procedure as in (c).

In [25]:
# create new BLP instance with first 100 markets
data_d1 = data[data[0] < 101]
data_d1 = create_data(data_d1, 1000000)
blp_d1 = BLP(data_d1, 1000)

In [26]:
%%time
beta_u = blp_d1.opt_beta()

CPU times: user 20 s, sys: 497 ms, total: 20.5 s
Wall time: 20.6 s


In [27]:
# estimated \beta_1^u
beta_u

     fun: 0.05742811141475579
    nfev: 39
     nit: 35
 success: True
       x: 0.6733044666502929

In [28]:
# run OLS regresion again, and get estimated \beta_0 and \beta_1^m
x = np.stack([np.ones(len(data_d1['Market ID'])),
                        np.asarray(data_d1.loc[:,'Characteristics'].values)],
                       axis=1)
y = np.log(data_d1.loc[:,'Market Share Ratio'].values) - np.log(data_d1.loc[:,'OutSide Share Ratio'].values)
res = np.linalg.solve(np.dot(x.T, x), np.dot(x.T, blp_d1.delta))
res

array([0.99922532, 1.94728431])

Therefore, estimated parameters are

\begin{eqnarray*}
    \beta_0 &=& 0.99922532 \\
    \beta_1^m &=& 1.94728431 \\
    \beta_1^u &=& 0.6733044666502929
\end{eqnarray*}

In [29]:
# create new BLP instance with first 10 markets
data_d2 = data[data[0] < 11]
data_d2 = create_data(data_d2, 1000000)
blp_d2 = BLP(data_d2, 1000)

In [30]:
%%time
beta_u = blp_d2.opt_beta()

CPU times: user 1.02 s, sys: 4.92 ms, total: 1.02 s
Wall time: 1.02 s


In [31]:
# estimated \beta_1^u
beta_u

     fun: 0.005198961024357971
    nfev: 20
     nit: 16
 success: True
       x: 0.27076139561843504

In [32]:
# run OLS regresion again, and get estimated \beta_0 and \beta_1^m
x = np.stack([np.ones(len(data_d2['Market ID'])),
                        np.asarray(data_d2.loc[:,'Characteristics'].values)],
                       axis=1)
y = np.log(data_d2.loc[:,'Market Share Ratio'].values) - np.log(data_d2.loc[:,'OutSide Share Ratio'].values)
res = np.linalg.solve(np.dot(x.T, x), np.dot(x.T, blp_d2.delta))
res

array([0.99698223, 1.96089766])

Therefore, estimated parameters are

\begin{eqnarray*}
    \beta_0 &=& 0.99698223 \\
    \beta_1^m &=& 1.96089766 \\
    \beta_1^u &=& 0.27076139561843504
\end{eqnarray*}

As we see from the estimations above, $\beta_0$ and $\beta_1^m$ remain same value, in addition these values are same with the estimation  in (a). This is because the mean utility level is same, so $\beta_1^m$ is not changed. Furthermore, the impact of outside option is not changed because the population in the market is same, so we have same $\beta_0$.
However, $\beta_1^u$ decreases as the number of sample markets decreases. $\beta_1^u$ indicates the standard deviation of the preference parameter. Therefore, changes in $\beta_1^u$ implies that the small number of sample size leads to underestimated value.